# 平均に隠れた本音を暴く。混合モデルによる顧客満足度構造の深堀り
「満足度 3.5 点」という平均値の裏側には、「熱狂的なファン」と「静かな不満層」が混在している可能性がある。この「声なき多層構造」をベイズの千里眼で可視化し、商品・サービス改善の真の優先順位を特定する。

## ユースケース
新商品発売やリニューアル後、アンケート結果が「中途半端な平均値」に落ち着いてしまい、次の一手に迷っている。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")

In [2]:
# Color Identity
COLOR_PURPLE = "#985DE5"
COLOR_BLUE   = "#118AB2"
COLOR_GREEN  = "#06D6A0"
COLOR_YELLOW = "#F9C74F"
COLOR_RED    = "#EF476F"
COLOR_GRAY   = "#8D99AE"

plt.rcdefaults()  # plt の現在のカラー定義をリセット
palette_brand = [COLOR_PURPLE, COLOR_BLUE, COLOR_GREEN, COLOR_YELLOW, COLOR_RED, COLOR_GRAY]

sns.set_theme(style="whitegrid", palette=palette_brand)
plt.rcParams["axes.prop_cycle"] = plt.cycler(color=palette_brand)

In [3]:
# 可視化時に日本語を表示可能にする
import japanize_matplotlib

plt.rcParams["font.family"] = "IPAexGothic"

In [4]:
# n=30 の満足度データを生成（熱狂層: 0.6, 失望層: 0.4）
np.random.seed(42)

n_obs = 30
idx_group = np.random.choice([0, 1], size=n_obs, p=[0.6, 0.4])
mu_true = [4.5, 1.8]
sigma_true = [0.3, 0.5]

obs_satisfaction = np.random.normal(loc=np.array(mu_true)[idx_group], scale=np.array(sigma_true)[idx_group])
obs_satisfaction = np.clip(obs_satisfaction, 1.0, 5.0)

In [10]:
obs_satisfaction.mean()  # 観測データの平均を出力

np.float64(3.600766062133175)

## ビジネス課題の定義
### 課題
- 「平均 3.5 点」という数字では、改善すべき点が「全員にとってのあと一歩」なのか、「一部の深刻な不満」なのか判別できない。
- 全員の平均を取ることで、熱狂的なファンの尖った評価が埋もれ、凡庸な施策に陥るリスクがある。

### 解決したい「問い」
- 満足度データの中に、異なる評価軸を持つ「隠れたグループ」はいくつ存在するか？
- 各グループの構成比率と、それぞれの平均的な満足度はどの程度か？